# Working with proxies in EF Core
Link to article: https://learn.microsoft.com/en-us/ef/ef6/fundamentals/proxies

## What is a proxy? 
When creating instances of POCO entity types, Entity Framework often creates instances of a dynamically generated derived type that acts as a proxy for the entity. This proxy overrides some virtual properties of the entity to insert hooks for performing actions automatically when the property is accessed.

In [1]:
#r "nuget: Microsoft.EntityFrameworkCore, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.Proxies, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.InMemory, 7.0.0"

Installed Packages Microsoft.EntityFrameworkCore, 7.0.0 Microsoft.EntityFrameworkCore.InMemory, 7.0.0 Microsoft.EntityFrameworkCore.Proxies, 7.0.0

In [7]:
using Microsoft.EntityFrameworkCore;
using Microsoft.Extensions.Logging;
using System;
using System.Reflection;

public class Post {
    public int Id {get; set;}
}

### DB Context without proxies:

In [3]:
public class DefaultContext : DbContext
{
    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder) {
        optionsBuilder
            .UseInMemoryDatabase("def");
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder) {
        base.OnModelCreating(modelBuilder);

        modelBuilder.Entity<Post>().HasData(
            new Post { Id = 1},
            new Post { Id = 2}
        );
    }
    public virtual DbSet<Post> Posts { get; set; }
}

{
    using DefaultContext context = new DefaultContext();
    context.Database.EnsureCreated();
    
    var post = context.Posts.FirstOrDefault();

    Console.WriteLine(post.GetType().Name ?? "No post found");
}

Post


### DB context with proxies:

In [12]:
public class LazyLoadedContext : DbContext
{
    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder) {
        optionsBuilder
            .UseLazyLoadingProxies()
            .UseInMemoryDatabase("lazy");
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder) {
        base.OnModelCreating(modelBuilder);

        modelBuilder.Entity<Post>().HasData(
            new Post { Id = 1},
            new Post { Id = 2}
        );
    }
    public virtual DbSet<Post> Posts { get; set; }
}

{
    using LazyLoadedContext context = new LazyLoadedContext();
    context.Database.EnsureCreated();
    
    var post = context.Posts.FirstOrDefault();

    Console.WriteLine($"Proxy type: {post.GetType().Name}, Base type: {post.GetType().BaseType.Name}");
    

}

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)